# Data Cleaning

> In this notebook we will be blah blah blah

---

## Imports

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from colorthief import ColorThief


from PIL import Image
import requests
from io import BytesIO

---

## Reading in Data

In [15]:
df = pd.read_csv('../data/raw_data.csv')
df

,Style,Artwork,Artist,Date,Link
0,Early-Dynastic,Narmer Palette,Ancient Egypt,3050 BC,https://uploads3.wikiart.org/00265/images/anci...
1,Early-Dynastic,Box Inlay with a Geometric Pattern,Ancient Egypt,3100-2900 BC,https://uploads2.wikiart.org/00244/images/anci...
2,Old-Kingdom,Khafre Enthroned,Ancient Egypt,2570 BC,https://uploads2.wikiart.org/00305/images/anci...
3,Middle-Kingdom,Stele of the Serpent King (Stela of Djet),Ancient Egypt,3000 BC,https://uploads7.wikiart.org/00305/images/anci...
4,Middle-Kingdom,"Laden Donkeys and Ploughing, Tomb of Djar",Ancient Egypt,2060-2010 BC,https://uploads8.wikiart.org/00244/images/anci...
...,...,...,...,...,...
124102,Street-Photography,Portrait of the corn stalk,Alfred Freddy Krupa,2019,https://uploads5.wikiart.org/00241/images/alfr...
124103,Street-Photography,The other side of life,Alfred Freddy Krupa,2019,https://uploads7.wikiart.org/00241/images/alfr...
124104,Street-Photography,The bonfire during construction,Alfred Freddy Krupa,2019,https://uploads7.wikiart.org/00242/images/alfr...
124105,Street-Photography,Limpidity,Alfred Freddy Krupa,2019,https://uploads7.wikiart.org/00248/images/alfr...


## Cleaning Data Functions

> This function will clean the data by step1, step2, step3. It takes in such and such argument and returns such and such value.

In [16]:
# Function that takes in Dataframe, returns numeric values for dates
def to_date(dataframe):

    #This converts all roman numbers to century
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XIX-XX cent.'], value='1800')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XX cent.'],value='1900')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XVIII cent.'],value='1700')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XIX cent.'],value='1800')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XX-XXI cent.'],value='1900')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XVI-XVII cent.'],value='1500')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XV-XVI cent.'],value='1400')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XVII-XVIII cent.'],value='1600')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XVI cent.'],value='1500')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XV cent.'],value='1400')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XIV-XV cent.'],value='1300')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XVII cent.'],value='1600')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XII-XIII cent.'],value='1100')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['X cent.'],value='900')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XIII-XIV cent.'],value='1200')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['VIII cent.'],value='700')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['VII-VIII cent.'],value='600')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XIV cent.'],value='1300')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XI cent.'],value='1000')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XI-XII cent.'],value='1000')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XXI cent.'],value='2000')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XIX-XX cent.'],value='1800')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['XVIII-XIX cent.'],value='1800')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['47'], value='1447')
    dataframe['Date'] = dataframe['Date'].replace(to_replace=['48'],value='1448')

    ## Hyphen remover
    for index, row in dataframe.iterrows():
        date_range = row['Date']
        for c in date_range:
            if c == '-':
                val = date_range[:date_range.index(c)]
                dataframe.at[index, 'Date'] = (val)
                break

    #This converts all date's to numeric values
    dataframe['Date'] = dataframe['Date'].astype(int)

    return dataframe

In [17]:
def cleaned_data(dataframe):
    """
    cleaned_data is a function that does blah blah blah and returns blah blah blah
    
    args: 
        dataframe: the dataframe that the user wants to clean
    """
    print('Before Cleaning')
    print('='*20)
    print('Columns')
    print(dataframe.columns)
    print('_'*20)
    print('Dataframe Size')
    print(dataframe.shape)
    print('_'*20)
    print('Dataframe Unique Values')
    print(dataframe.nunique())
    print('_'*20)
    print('Null Values in Each Column')
    print(dataframe.isna().sum())
    print('_'*20)
    print('Data Types in Each Column')
    print(dataframe.dtypes)
    print('='*20)
    
    #This drops cuplicates of any work of art to decrease chances of sketches
    dataframe = dataframe.drop_duplicates(subset=['Artwork', 'Artist', 'Date'], 
                       keep='last')
    
    #This drops any values in the style columns that have less than a count of 500
    #So the model can properly train on the style
    s = dataframe['Style'].value_counts() > 500
    s = list(s[s == True].index)
    dataframe = dataframe[dataframe['Style'].isin(s)]
    
    dataframe = to_date(dataframe)
    
    print('After Cleaning')
    print('='*20)
    print('Columns')
    print(dataframe.columns)
    print('_'*20)
    print('Dataframe Size')
    print(dataframe.shape)
    print('_'*20)
    print('Dataframe Unique Values')
    print(dataframe.nunique())
    print('_'*20)
    print('Null Values in Each Column')
    print(dataframe.isna().sum())
    print('_'*20)
    print('Data Types in Each Column')
    print(dataframe.dtypes)
    print('='*20)
    
    #returning the dataframe with an index reset
    return dataframe.reset_index(drop=True)

In [18]:
clean_df = cleaned_data(df)
clean_df

Before Cleaning
Columns
Index(['Style', 'Artwork', 'Artist', 'Date', 'Link'], dtype='object')
____________________
Dataframe Size
(124107, 5)
____________________
Dataframe Unique Values
Style         217
Artwork     92007
Artist       3052
Date         2958
Link       116657
dtype: int64
____________________
Null Values in Each Column
Style      0
Artwork    0
Artist     0
Date       0
Link       0
dtype: int64
____________________
Data Types in Each Column
Style      object
Artwork    object
Artist     object
Date       object
Link       object
dtype: object
After Cleaning
Columns
Index(['Style', 'Artwork', 'Artist', 'Date', 'Link'], dtype='object')
____________________
Dataframe Size
(89637, 5)
____________________
Dataframe Unique Values
Style         51
Artwork    76133
Artist      2632
Date         621
Link       89637
dtype: int64
____________________
Null Values in Each Column
Style      0
Artwork    0
Artist     0
Date       0
Link       0
dtype: int64
____________________
Dat

,Style,Artwork,Artist,Date,Link
0,Early-Renaissance,Virgen Con El Niño,Álvaro Pires de Évora,1447,https://uploads6.wikiart.org/00312/images/alva...
1,Early-Renaissance,A Virgem em Majestade,Álvaro Pires de Évora,1448,https://uploads3.wikiart.org/00312/images/alva...
2,Early-Renaissance,"The Sacrifice of Isaac, bronze competition rel...",Filippo Brunelleschi,1401,https://uploads8.wikiart.org/images/filippo-br...
3,Early-Renaissance,Madonna with Child,Filippo Brunelleschi,1402,https://uploads2.wikiart.org/images/filippo-br...
4,Early-Renaissance,The sacrifice of Isaac,Lorenzo Ghiberti,1401,https://uploads0.wikiart.org/00164/images/ghib...
...,...,...,...,...,...
89632,Ukiyo-e,Lucy's Mutation,Takato Yamamoto,2014,https://uploads3.wikiart.org/00147/images/taka...
89633,Ukiyo-e,Common Sense,Takato Yamamoto,2014,https://uploads3.wikiart.org/00147/images/taka...
89634,Ukiyo-e,Sacred Circulation,Takato Yamamoto,2015,https://uploads3.wikiart.org/00147/images/taka...
89635,Ukiyo-e,Ecstasy of Linked Circles,Takato Yamamoto,2015,https://uploads3.wikiart.org/00147/images/taka...


# Creating Art Image Folders

In [19]:
# root_path = '../images/styles'
# list_styles = [i for i in clean_df['Style'].unique()]
# for items in list_styles:
#     path = os.path.join(root_path, items)
#     os.mkdir(path)

# Putting Images in Folders

In [20]:
# Daniel's modified scrape to run on separate kernels at same time
kernel_num = 0  #change this one. [0, 1, 2, ..., 19]
total_kernels = 20
splits=np.round(np.linspace(0, clean_df.shape[0], num=total_kernels+1)).astype(int)
start_index=splits[kernel_num]
stop_index=splits[kernel_num+1]
for i, row in clean_df.iloc[start_index:stop_index].iterrows():
   response = requests.get(clean_df['Link'][i])
   img = Image.open(BytesIO(response.content))
   print(clean_df['Style'][i])
   print(i)
   print(img.mode)
   img = img.convert("RGB")
   img = img.resize((250,250))
   filepath = f'../images/styles/{clean_df["Style"][i]}/image_{i}.jpg'
   img.save(filepath)

Early-Renaissance
0
RGB
Early-Renaissance
1
RGB
Early-Renaissance
2
RGB
Early-Renaissance
3
RGB
Early-Renaissance
4
RGB
Early-Renaissance
5
RGB
Early-Renaissance
6
RGB
Early-Renaissance
7
RGB
Early-Renaissance
8
RGB
Early-Renaissance
9
RGB
Early-Renaissance
10
RGB
Early-Renaissance
11
RGB
Early-Renaissance
12
RGB
Early-Renaissance
13
RGB
Early-Renaissance
14
RGB
Early-Renaissance
15
RGB
Early-Renaissance
16
RGB
Early-Renaissance
17
RGB
Early-Renaissance
18
RGB
Early-Renaissance
19
RGB
Early-Renaissance
20
RGB
Early-Renaissance
21
RGB
Early-Renaissance
22
RGB
Early-Renaissance
23
RGB
Early-Renaissance
24
RGB
Early-Renaissance
25
RGB
Early-Renaissance
26
RGB
Early-Renaissance
27
RGB
Early-Renaissance
28
RGB
Early-Renaissance
29
RGB
Early-Renaissance
30
RGB
Early-Renaissance
31
RGB
Early-Renaissance
32
RGB
Early-Renaissance
33
RGB
Early-Renaissance
34
RGB
Early-Renaissance
35
RGB
Early-Renaissance
36
RGB
Early-Renaissance
37
RGB
Early-Renaissance
38
RGB
Early-Renaissance
39
RGB
Early-Rena

In [21]:
clean_df

,Style,Artwork,Artist,Date,Link
0,Early-Renaissance,Virgen Con El Niño,Álvaro Pires de Évora,1447,https://uploads6.wikiart.org/00312/images/alva...
1,Early-Renaissance,A Virgem em Majestade,Álvaro Pires de Évora,1448,https://uploads3.wikiart.org/00312/images/alva...
2,Early-Renaissance,"The Sacrifice of Isaac, bronze competition rel...",Filippo Brunelleschi,1401,https://uploads8.wikiart.org/images/filippo-br...
3,Early-Renaissance,Madonna with Child,Filippo Brunelleschi,1402,https://uploads2.wikiart.org/images/filippo-br...
4,Early-Renaissance,The sacrifice of Isaac,Lorenzo Ghiberti,1401,https://uploads0.wikiart.org/00164/images/ghib...
...,...,...,...,...,...
89632,Ukiyo-e,Lucy's Mutation,Takato Yamamoto,2014,https://uploads3.wikiart.org/00147/images/taka...
89633,Ukiyo-e,Common Sense,Takato Yamamoto,2014,https://uploads3.wikiart.org/00147/images/taka...
89634,Ukiyo-e,Sacred Circulation,Takato Yamamoto,2015,https://uploads3.wikiart.org/00147/images/taka...
89635,Ukiyo-e,Ecstasy of Linked Circles,Takato Yamamoto,2015,https://uploads3.wikiart.org/00147/images/taka...


## Saving the Cleaned Dataframe

> We save the data as blah blah so that in later notebooks it can be used for eda and modeling prurposes much easier.

In [22]:
clean_df.to_csv('../data/clean_art.csv', index = False)

## Get Color Data

In [23]:
# for i, row in clean_df.iterrows():
#     response = requests.get(clean_df['Link'][i])
#     img = Image.open(BytesIO(response.content))
#     filepath = f'../images/styles/{clean_df["Style"][i]}/image_{i}.jpg'
#     color_thief = ColorThief(filepath)
#     clean_df['dom_color'][i] = color_thief.get_color(quality=1)